# Classification trees

## Intro

Classification trees are one type of **decision tree** that can be used for predicting categorical outputs.
In the next notebook we will look at regression trees, which are used for making continuous predictions.

If you have ever played the game 21 questions then you already know what a classification tree is.
Or if you've ever tried to code up an artificial general intelligence using a lot of `if` statements, then you've essentially built up your own decision tree by hand.

These examples use the obvious approach of building a classifier by asking a series of yes/no questions.
The answer to each question gives you more info on class of the object until the range of possible options becomes fine enough for you to settle on a prediction.

Below is an example of such a classifier.
This one predicts the gender using the height and weight of an example person.

![](images/CARTS_example_tree.png)

The above examples however, depend on you finding the right questions to ask manually.
We'd like to be able to find the right questions to ask automatically, using machine learning.
This means that we need to start quantifying what makes a **good** split.

![](images/CARTS_geni_impurity.png)

![](images/CARTS_split_pos.png)

Let's build a classification tree from scratch

In [ ]:
import numpy as np

def region_impurity(examples_in_region):
    X, Y = examples_in_region
    n_classes = np.ptp(Y)
    n_datapoints = len(X)
    print('n_classes:', n_classes)
    impurity = 0
    for class_idx in range(n_classes):
        n_examples_in_class = len(X[Y == class_idx])
        p_k = examples_in_class / n_datapoints
        impurity += p_k * (1 - p_k)
    return impurity

def split_impurity(examples_split, split_pos):


def get_split_candidates(X):
    return (X[:-1] + X[1:]) / 2

def get_best_split_pos(X, split_candidates):

def train(X, Y, max_depth=4):
    done = False
    depth = 0
    while not done:
        split_candidates = get_split_candidates(X)
        best_split_pos = get_best_split_pos(split_candidates)
        


    